In [60]:
from selenium import webdriver
# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By
# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點擊、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains
# 加入鍵盤功能 (例如 Ctrl、Alt 等)
from selenium.webdriver.common.keys import Keys
# 強制等待 (執行期間休息一下)
from time import sleep
import requests as req
import os
import ddddocr
from selenium.webdriver.support.ui import Select
import random
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行

my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36")
my_options.add_experimental_option("excludeSwitches", ["enable-automation"])
my_options.add_experimental_option('useAutomationExtension', False)



In [81]:
# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options
)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        });
    """
})
def human_like_pause(min_sec=0.8, max_sec=2.5):
    #模擬人類行為的隨機等待
    sleep(random.uniform(min_sec, max_sec))

def openWebsite():
    # 使用 Chrome 的 WebDriver
    driver.get('https://www.thsrc.com.tw/')
    accpet_button = driver.find_element(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')
    ac = ActionChains(driver)
    human_like_pause()
    ac.move_to_element(accpet_button)
    ac.click(accpet_button).perform()
    human_like_pause()

    orderWindow = driver.find_element(By.CSS_SELECTOR, 'li.nav-item > a#a_irs')
    ac.move_to_element(orderWindow)
    human_like_pause()
    ac.click(orderWindow)
    ac.perform()

def intoOrderWeb():
    # start_staion = driver.find_element(By.CSS_SELECTOR, 'select[name=selectStartStation][id=select_location01]')
    iframe = driver.find_element(By.CSS_SELECTOR, 'iframe#irsUrl')
    iframeSrc = iframe.get_attribute('src')
    driver.get(f"{iframeSrc}")

def startStaion():
    start_station = driver.find_element(By.CSS_SELECTOR, 'select#select_location01')
    start_sta = Select(start_station)
    start_sta.select_by_value("1")

def arriveStation():
    arrive_station = driver.find_element(By.CSS_SELECTOR, 'select#select_location02')
    start_sta = Select(arrive_station)
    start_sta.select_by_value("12")

def goDate():
    go_date = driver.find_element(By.CSS_SELECTOR, 'input#Departdate02')
    # go_date.clear()
    go_date.send_keys(Keys.CONTROL, "a")
    go_date.send_keys('2025/06/11')

def goTime():
    go_time = driver.find_element(By.CSS_SELECTOR, 'input#toPortalTimeTable')
    # go_date.clear()
    go_time.send_keys(Keys.CONTROL, "a")
    go_time.send_keys('18:00')   

def securityCode():
    #抓取識別碼圖片
    securityCodeImg = driver.find_element(By.CSS_SELECTOR, "img#BookingS1Form_homeCaptcha_passCode")
    #截圖識別碼
    img_bytes = securityCodeImg.screenshot_as_png
    #利用ddddocr來抓取圖片內的英數字
    ocr = ddddocr.DdddOcr()
    result = ocr.classification(img_bytes)
    #輸入識別碼
    securityCode = driver.find_element(By.CSS_SELECTOR, "input#securityCode")
    securityCode.clear()
    securityCode.send_keys(result)

def search():
    searchBtn = driver.find_element(By.CSS_SELECTOR, 'input#SubmitButton')
    ac = ActionChains(driver)
    ac.move_to_element(searchBtn)
    human_like_pause()
    ac.click(searchBtn)
    ac.perform()

def checkTime():
    checkBtn = driver.find_element(By.CSS_SELECTOR, 'input[name=SubmitButton]')
    
    ac = ActionChains(driver)
    ac.move_to_element(checkBtn)
    human_like_pause()
    ac.click(checkBtn).perform()

    

In [82]:
openWebsite()
intoOrderWeb()
startStaion()
arriveStation()
goDate()
goTime()
securityCode()
search()
checkTime()

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i


In [ ]:
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏
        例如將上方的 script 改成:
        offset += 600
        '''

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        
        '''
        [補充]
        如果要滾動的是 div 裡面的捲軸，可以使用以下的方法
        document.querySelector('div').scrollTo({...})
        '''
        
        # (重要)強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(3)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
            
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 600:
            break